# Calibrating the Traffic

After all schedules have been added to the database, now the distribution for the vehicle type of the onward transportation of the container need to be be calibrated accordingly.
This distribution decides which vehicle type will pick up the container on its outbound journey.
Calibrating this distribution to your input data is good practice as it gives you the chance to re-check your assumptions regarding the traffic.
Technically speaking the distribution will be automatically adjusted during the generation process if required.
It will issue a warning when doing so.
In this Jupyter Notebook, it is show how the calibration can be done.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from conflowgen import DatabaseChooser
from conflowgen import ContainerFlowGenerationManager
from conflowgen import InboundAndOutboundVehicleCapacityPreviewReport
from conflowgen import ContainerFlowByVehicleTypePreviewReport
from conflowgen import VehicleCapacityExceededPreviewReport
from conflowgen import ModalSplitPreviewReport
from conflowgen import ModeOfTransport

## Load database

Load information from database.

In [ ]:
database_chooser = DatabaseChooser()
demo_file_name = "demo_deham_cta.sqlite"
if demo_file_name in database_chooser.list_all_sqlite_databases():
    database_chooser.load_existing_sqlite_database(demo_file_name)
else:
    print("Database is missing, nothing to do here")

Get information regarding the container flow.

In [ ]:
container_flow_generation_manager = ContainerFlowGenerationManager()
container_flow_properties = container_flow_generation_manager.get_properties()
for key, value in container_flow_properties.items():
    print(f"{key:<60}: {value}")

## Load text reports

Generate text reports.
These have been tested with unit tests and serve as guidance for the visualizations.

In [ ]:
inbound_and_outbound_vehicle_capacity_preview_report = InboundAndOutboundVehicleCapacityPreviewReport()
report = inbound_and_outbound_vehicle_capacity_preview_report.get_report_as_text()
print("Inbound and outbound traffic: ")
print(report)
print()

container_flow_by_vehicle_type_preview_report = ContainerFlowByVehicleTypePreviewReport()
report = container_flow_by_vehicle_type_preview_report.get_report_as_text()
print("Container flow between vehicle types as defined by schedules and distributions: ")
print(report)
print()

modal_split_preview_report = ModalSplitPreviewReport()
report = modal_split_preview_report.get_report_as_text()
print("The same container flow expressed in terms of transshipment and modal split for the hinterland: ")
print(report)
print()

vehicle_capacity_exceeded_preview_report = VehicleCapacityExceededPreviewReport()
report = vehicle_capacity_exceeded_preview_report.get_report_as_text()
print("Consequences of container flow for outgoing vehicles: ")
print(report)

## Plot inbound and outbound capacities

In [ ]:
inbound_and_outbound_vehicle_capacity_preview_report.get_report_as_graph()
sns.set_palette(sns.color_palette())
plt.show()

If the outbound capacity is the same like the maximum capacity, it means all capacities are used.
This can either happen if the transport buffer is 0%.

## Plot intended container flows

In [ ]:
fig = container_flow_by_vehicle_type_preview_report.get_report_as_graph()
fig.show()

## Plot capacity exceeded

In [ ]:
vehicle_capacity_exceeded_preview_report.get_report_as_graph()
sns.set_palette(sns.color_palette())
plt.show()

If a bar of the currently planned value exceeds the maximum, this means that there is a mismatch - there are more transports planned for the outbound journey of that vehicle than there is capacity left on these vehicles.

## Visualize transshipment share and modal split for hinterland

In [ ]:
modal_split_preview_report.get_report_as_graph()
sns.set_palette(sns.color_palette())
plt.show()

## Combine visuals for tuning

Here, you can adjust the mode of transport distribution and directly get the preview of how this will affect the container flow.
Be aware that this is just a preview.

For the CTA example, the calibration was based on the information available on 
https://www.hafen-hamburg.de/de/statistiken/modal-split/ during September 2021.
At that time, 3 mio TEU were transshipped from vessel to vessel and 5.5 mio TEU were transshipped to or from the hinterland,
corresponding to 35% were transshipment.
Regarding the modal split for the hinterland, in September 20217% for train, 50.2% for trucks, and 2.8% for barges were reported.

In [ ]:
hypothesized_mode_of_transport_distribution = {
    ModeOfTransport.truck: {
        ModeOfTransport.truck: 0,
        ModeOfTransport.train: 0,
        ModeOfTransport.barge: 0,
        ModeOfTransport.feeder: 0.8 / (0.8 + 4.6) + 0.15,
        ModeOfTransport.deep_sea_vessel: 4.6 / (0.8 + 4.6) - 0.15
    },
    ModeOfTransport.train: {
        ModeOfTransport.truck: 0,
        ModeOfTransport.train: 0,
        ModeOfTransport.barge: 0,
        ModeOfTransport.feeder: 0.8 / (0.8 + 4.6) + 0.15,
        ModeOfTransport.deep_sea_vessel: 4.6 / (0.8 + 4.6) - 0.15
    },
    ModeOfTransport.barge: {
        ModeOfTransport.truck: 0,
        ModeOfTransport.train: 0,
        ModeOfTransport.barge: 0,
        ModeOfTransport.feeder: 0.8 / (0.8 + 4.6),
        ModeOfTransport.deep_sea_vessel: 4.6 / (0.8 + 4.6)
    },
    ModeOfTransport.feeder: {
        ModeOfTransport.truck: 0.8 / (0.8 + 1.9) * 0.502,
        ModeOfTransport.train: 0.8 / (0.8 + 1.9) * 0.47,
        ModeOfTransport.barge: 0.8 / (0.8 + 1.9) * 0.0028,
        ModeOfTransport.feeder: 0,
        ModeOfTransport.deep_sea_vessel: 1.9 / (0.8 + 1.9)
    },
    ModeOfTransport.deep_sea_vessel: {
        ModeOfTransport.truck: 4.6 / (4.6 + 1.9) * 0.502,
        ModeOfTransport.train: 4.6 / (4.6 + 1.9) * 0.47,
        ModeOfTransport.barge: 4.6 / (4.6 + 1.9) * 0.0028,
        ModeOfTransport.feeder: 1.9 / (4.6 + 1.9),
        ModeOfTransport.deep_sea_vessel: 0
    }
}

container_flow_by_vehicle_type_preview_report.hypothesize_with_mode_of_transport_distribution(hypothesized_mode_of_transport_distribution)
fig = container_flow_by_vehicle_type_preview_report.get_report_as_graph()
fig.show()

inbound_and_outbound_vehicle_capacity_preview_report.hypothesize_with_mode_of_transport_distribution(hypothesized_mode_of_transport_distribution)
inbound_and_outbound_vehicle_capacity_preview_report.get_report_as_graph()
sns.set_palette(sns.color_palette())
plt.show()

vehicle_capacity_exceeded_preview_report.hypothesize_with_mode_of_transport_distribution(hypothesized_mode_of_transport_distribution)
vehicle_capacity_exceeded_preview_report.get_report_as_graph()
sns.set_palette(sns.color_palette())
plt.show()

modal_split_preview_report.hypothesize_with_mode_of_transport_distribution(hypothesized_mode_of_transport_distribution)
modal_split_preview_report.get_report_as_graph()
plt.show()